In [ ]:
!pip install idx2numpy
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My\ Drive/Thesis_data/

/content/drive/My Drive/Thesis_data


In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
from collections import defaultdict
import gzip
import cv2 as cv
import os
import numpy as np
import random

TensorFlow 1.x selected.


In [ ]:
def extract_data():
    train_labels = []
    test_labels = []
    label_names = []
    IMG_SIZE = 128
    IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
    train_np = np.empty((0,128,128,3), float)
    test_np = np.empty((0,128,128,3), float)
    train_features = np.empty((0,4,4,512), float)
    test_features = np.empty((0,4,4,512), float)

    np_train_path = './numpy_files_RGB-D/train'
    np_test_path = './numpy_files_RGB-D/test'
    np_train_label_path = './numpy_files_RGB-D/train_labels.npy'
    np_test_label_path = './numpy_files_RGB-D/test_labels.npy'
    np_label_names_path = './numpy_files_RGB-D/label_names.npy'

    vgg = tf.contrib.keras.applications.VGG16(input_shape=IMG_SHAPE, weights="imagenet", include_top=False)
    vgg.trainable = False
    
    print("\n..........loading dataset from numpy files..........\n")

    for i in range(1,78):
      with gzip.GzipFile(np_train_path+str(i)+".npy.gz", "r") as f:
        train_np = np.load(f)
        train_features = np.append(train_features, vgg.predict(train_np, batch_size=32), axis=0)
        train_np = None

    print(train_features.shape)

    for i in range(1,21):
      with gzip.GzipFile(np_test_path+str(i)+".npy.gz", "r") as f:
        test_np = np.load(f)
        test_features = np.append(test_features, vgg.predict(test_np, batch_size=32), axis=0)
        test_np = None

    print(test_features.shape)

    train_labels = np.load(np_train_label_path)
    test_labels = np.load(np_test_label_path)
    label_names = np.load(np_label_names_path)

    train_features_flatten = train_features.reshape((train_features.shape[0], train_features.shape[1] * train_features.shape[2] * train_features.shape[3]))

    test_features_flatten = test_features.reshape((test_features.shape[0], test_features.shape[1] * test_features.shape[2] * test_features.shape[3]))

    return train_features_flatten, train_labels, test_features_flatten, test_labels, label_names

In [ ]:
if __name__ == "__main__":
    
    f_train_path = './extracted_features_RGB-D/VGG16/train.npy.gz'
    f_test_path = './extracted_features_RGB-D/VGG16/test.npy.gz'
    f_train_label_path = './extracted_features_RGB-D/VGG16/train_labels.npy'
    f_test_label_path = './extracted_features_RGB-D/VGG16/test_labels.npy'
    f_label_names_path = './extracted_features_RGB-D/VGG16/label_names.npy'

    train_data, train_labels, test_data, test_labels, label_names = extract_data()

    os.makedirs(os.path.dirname(f_train_path), exist_ok=True)

    with gzip.GzipFile(f_train_path, "w") as f:
        np.save(f, train_data)
    with gzip.GzipFile(f_test_path, "w") as f:
        np.save(f, test_data)

    np.save(f_train_label_path, train_labels)
    np.save(f_test_label_path, test_labels)
    np.save(f_label_names_path, label_names)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

..........loading dataset from numpy files..........

(38262, 4, 4, 512)
(9503, 4, 4, 512)
